# Determine expression relationships between constructs
Correlate post feature selection well-aggregated morphology features across the same concentrations.

## Imports

In [ ]:
import pathlib
import sys

import pandas as pd

## Find the root of the git repo on the host system

In [ ]:
# Get the current working directory
cwd = pathlib.Path.cwd()

if (cwd / ".git").is_dir():
    root_dir = cwd

else:
    root_dir = None
    for parent in cwd.parents:
        if (parent / ".git").is_dir():
            root_dir = parent
            break

# Check if a Git root directory was found
if root_dir is None:
    raise FileNotFoundError("No Git root directory found.")

## Define paths

### Input paths

In [ ]:
# Path to correlation class
sys.path.append(
    f"{root_dir}/0.data_analysis/utils"
)

# Class for calculating correlations
from CorrelateData import CorrelateData

platedf_path = pathlib.Path(root_dir / "nf1_painting_repo/3.processing_features/data/single_cell_profiles/Plate_4_bulk_camerons_method.parquet").resolve(strict=True)
platedf = pd.read_parquet(platedf_path)

### Output paths

In [ ]:
data_path = pathlib.Path("construct_correlation_data")
data_path.mkdir(parents=True, exist_ok=True)

## Label untreated cells

In [ ]:
platedf["Metadata_siRNA"].fillna("No Construct", inplace=True)
platedf.dropna(inplace=True)

In [ ]:
meta_cols = platedf.filter(like="Metadata").columns
feat_cols = platedf.drop(columns=meta_cols).columns

## Compute Correlations

In [ ]:
# Store correlations
corrdfs = []

cp = CorrelateData()

# Include cells with no construct treatment
platedfz = platedf.loc[platedf["Metadata_Concentration"] == 0.0].copy()

# Compute correlations for each concentration
for conc, concdf in platedf.groupby("Metadata_Concentration"):

    # Include the cells not treated with a construct in the correlation comparisons
    concdf = pd.concat([
        concdf,
        platedfz.copy()
    ], axis=0)

    corrdfs.append(cp.intra_correlations(
        _df = concdf.reset_index(drop=True).copy(),
        _antehoc_group_cols = ["Metadata_siRNA", "Metadata_genotype"],
        _feat_cols = feat_cols,
        _posthoc_group_cols = ["Metadata_Well"],
    )
    )

    # Save the concentration and type of comparison
    corrdfs[-1]["Metadata_Concentration"] = conc
    corrdfs[-1]["cell_comparison"] = "intra_well"

    # Don't compute correlations for cells not treated with a construct
    # The cells in this group is already compared to the constructs at every other concentration
    if conc == 0.0:
        continue

    # Correlates all cells except those occupying the same well
    corrdfs.append(cp.inter_correlations(
        _df = platedf.reset_index(drop=True).copy(),
        _antehoc_group_cols = ["Metadata_siRNA"],
        _feat_cols = feat_cols,
        _posthoc_group_cols = ["Metadata_Well", "Metadata_genotype"],
    )
    )

    # Save the concentration and type of comparison
    corrdfs[-1]["Metadata_Concentration"] = conc
    corrdfs[-1]["cell_comparison"] = "inter_well"

## Store Correlation Data

In [ ]:
corrdfs = pd.concat(corrdfs, axis=0)
corrdfs.to_parquet(f"{data_path}/plate_4_sc_feature_selected_camerons_agg_well_correlations.parquet")

In [ ]:
corrdfs.head()